In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lead
from pyspark.sql.types import LongType

#from pyspark import SparkContext

In [2]:
spark = SparkSession.builder.appName("Preprocessing").getOrCreate()

In [31]:
raw_data_act = spark.read.parquet('activityObf.parquet')


In [ ]:
raw_data_act_int = raw_data_act.withColumn('user_id', raw_data_act['user_id'].cast(LongType()))

In [ ]:
df_act_wo_duplicates = raw_data_act_int.dropDuplicates()

In [ ]:
#df_act_wo_duplicates.write.csv("act_wo_duplicates.csv", header = True)

In [3]:
df_act_wo_duplicates = spark.read.csv("act_wo_duplicates.csv/", header = True, inferSchema = True)


In [4]:
#df_act_ordered = df_act_wo_duplicates.orderBy(['user_id', 'periodo'])

In [4]:
from pyspark.sql import Window

w = Window.orderBy(['user_id','periodo']) ## --> esto ordena por user_id y periodo

# df_act_orderer_w_lead = df_act_ordered.withColumn("next_periodo", lead(df_act_ordered.periodo).over(w))\
#             .withColumn("next_user_id", lead(df_act_ordered.user_id).over(w))

df_act_orderer_w_lead = df_act_wo_duplicates.withColumn("next_periodo", lead(df_act_wo_duplicates.periodo).over(w))\
            .withColumn("next_user_id", lead(df_act_wo_duplicates.user_id).over(w))

In [41]:
## Para merge de df's de act. y sb, ver --> https://stackoverflow.com/questions/38063657/pyspark-merge-outer-join-two-data-frames

91371067

In [5]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import (month, year)
from pyspark.sql.functions import udf, from_unixtime, unix_timestamp

def estado_actividad(u_id, nxt_u_id, per, nxt_per):
    
    #def month_diff(x,y):
        
    #    return 12*(year(y)[0] - year(x)[0]) + (month(y)[0] - month(x)[0])
    
    if u_id == nxt_u_id and 12*(nxt_per.year - per.year) + (nxt_per.month - per.month) >= 4:#12*(year(nxt_per) - year(per)) + (month(nxt_per) - month(per)) >= 4:
        
        return 1
    else:
        return 0
    


In [6]:
define_estados = udf(estado_actividad, IntegerType())    

# from_pattern = 'yyyy-MM-dd hh:mm:ss'
# to_pattern = 'yyyy-MM-dd'

# df_act_orderer_w_lead = df_act_orderer_w_lead.withColumn("periodo", from_unixtime(unix_timestamp(df_act_orderer_w_lead.periodo, 
#             from_pattern), to_pattern)).withColumn("next_periodo", 
#             from_unixtime(unix_timestamp(df_act_orderer_w_lead.next_periodo, from_pattern), to_pattern))
   

In [7]:
df_act_final = df_act_orderer_w_lead.withColumn("estado", define_estados(df_act_orderer_w_lead.user_id,
                                                         df_act_orderer_w_lead.next_user_id,
                                                         df_act_orderer_w_lead.periodo,
                                                         df_act_orderer_w_lead.next_periodo)) 

In [8]:
df_act_final.show()

+-------------+-------------------+-------------------+-------------+------+
|      user_id|            periodo|       next_periodo| next_user_id|estado|
+-------------+-------------------+-------------------+-------------+------+
|3263522195314|2017-07-01 00:00:00|2017-08-01 00:00:00|3263522195314|     0|
|3263522195314|2017-08-01 00:00:00|2017-09-01 00:00:00|3263522195314|     0|
|3263522195314|2017-09-01 00:00:00|2017-10-01 00:00:00|3263522195314|     0|
|3263522195314|2017-10-01 00:00:00|2017-11-01 00:00:00|3263522195314|     0|
|3263522195314|2017-11-01 00:00:00|2017-12-01 00:00:00|3263522195314|     0|
|3263522195314|2017-12-01 00:00:00|2018-01-01 00:00:00|3263522195314|     0|
|3263522195314|2018-01-01 00:00:00|2018-02-01 00:00:00|3263522195314|     0|
|3263522195314|2018-02-01 00:00:00|2018-03-01 00:00:00|3263522195314|     0|
|3263522195314|2018-03-01 00:00:00|2018-04-01 00:00:00|3263522195314|     0|
|3263522195314|2018-04-01 00:00:00|2018-05-01 00:00:00|3263522195314|     0|

In [18]:
df_act_final.cache()

DataFrame[user_id: bigint, periodo: timestamp, next_periodo: timestamp, next_user_id: bigint, estado: int]

In [20]:
df_act_final.describe().show()

+-------+--------------------+--------------------+--------------------+
|summary|             user_id|        next_user_id|              estado|
+-------+--------------------+--------------------+--------------------+
|  count|            91371067|            91371066|            91371067|
|   mean|2.454699089645138...|2.454699116509949...|0.001333365188785636|
| stddev| 1.58195864893496E17|1.581958636749278...|  0.0364909213453438|
|    min|       3263522195314|       3263522195314|                   0|
|    max| 2197891185576109877| 2197891185576109877|                   1|
+-------+--------------------+--------------------+--------------------+



#### Dataframe sb

In [11]:
df_sb = spark.read.parquet('sbObf.parquet/')

In [12]:
df_sb.printSchema()

root
 |-- fecha_proceso: date (nullable = true)
 |-- R04PERREF: string (nullable = true)
 |-- periodo: date (nullable = true)
 |-- user_id: string (nullable = true)
 |-- total_deudores_informados: integer (nullable = true)
 |-- deuda_vigente: long (nullable = true)
 |-- deuda_directa_morosa90: long (nullable = true)
 |-- deuda_directa_vencida: long (nullable = true)
 |-- deuda_directa_mora180: long (nullable = true)
 |-- deuda_indirecta_mora180: long (nullable = true)
 |-- deuda_directa_inversiones_financieras: long (nullable = true)
 |-- deuda_directa_ops_pacto: long (nullable = true)
 |-- deuda_indirecta_vigente: long (nullable = true)
 |-- deuda_indirecta_vencida: long (nullable = true)
 |-- deuda_directa_comercial: long (nullable = true)
 |-- deuda_directa_cred_consumo: long (nullable = true)
 |-- deuda_directa_hipotecaria: long (nullable = true)
 |-- deuda_directa_comercial_ext: long (nullable = true)
 |-- deuda_indirecta_comercial_ext: long (nullable = true)
 |-- deuda_directa_le

In [14]:
df_sb_int = df_sb.withColumn('user_id', df_sb['user_id'].cast(LongType()))
df_sb_int.printSchema()

root
 |-- fecha_proceso: date (nullable = true)
 |-- R04PERREF: string (nullable = true)
 |-- periodo: date (nullable = true)
 |-- user_id: long (nullable = true)
 |-- total_deudores_informados: integer (nullable = true)
 |-- deuda_vigente: long (nullable = true)
 |-- deuda_directa_morosa90: long (nullable = true)
 |-- deuda_directa_vencida: long (nullable = true)
 |-- deuda_directa_mora180: long (nullable = true)
 |-- deuda_indirecta_mora180: long (nullable = true)
 |-- deuda_directa_inversiones_financieras: long (nullable = true)
 |-- deuda_directa_ops_pacto: long (nullable = true)
 |-- deuda_indirecta_vigente: long (nullable = true)
 |-- deuda_indirecta_vencida: long (nullable = true)
 |-- deuda_directa_comercial: long (nullable = true)
 |-- deuda_directa_cred_consumo: long (nullable = true)
 |-- deuda_directa_hipotecaria: long (nullable = true)
 |-- deuda_directa_comercial_ext: long (nullable = true)
 |-- deuda_indirecta_comercial_ext: long (nullable = true)
 |-- deuda_directa_leas

In [23]:
df_act_final.write.csv("df_act_final.csv", header = True)

In [24]:
df_sb_wo_duplicates = df_sb_int.dropDuplicates()

In [ ]:
df_sb_wo_duplicates.count()